In [2]:
import pandas as pd
import os
from pandasql import sqldf

#### Load the datas

In [57]:
employee = pd.read_csv('./data/employee.csv')
employee.sample(5)

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,HIRE_DATE,JOB_ID,SALARY,DEPARTMENT_ID,RESIDENCY_ID
17,108,Nancy,Greenberg,NGREENBE,17-Aug-02,FI_MGR,12008,100,1700
35,126,Irene,Mikkilineni,IMIKKILI,28-Sep-06,ST_CLERK,2700,50,2400
27,118,Guy,Himuro,GHIMURO,15-Nov-06,PU_CLERK,2600,30,1700
9,100,Steven,King,SKING,17-Jun-03,AD_PRES,24000,0,3000
18,109,Daniel,Faviet,DFAVIET,16-Aug-02,FI_ACCOUNT,9000,100,1700


In [3]:
employee.shape

(50, 9)

In [4]:
employee.columns

Index(['EMPLOYEE_ID', 'FIRST_NAME', 'LAST_NAME', 'EMAIL', 'HIRE_DATE',
       'JOB_ID', 'SALARY', 'DEPARTMENT_ID', 'RESIDENCY_ID'],
      dtype='object')

In [5]:
department = pd.read_csv('./data/department.csv')
department.sample(5)

,DEPARTMENT_ID,DEPARTMENT_NAME,MANAGER_ID,LOCATION_ID
0,10,Administration,200.0,1700
7,80,Sales,145.0,2500
4,50,Shipping,121.0,2000
8,90,Executive,100.0,1700
2,30,Purchasing,114.0,1700


In [6]:
department.shape

(12, 4)

In [7]:
department.columns

Index(['DEPARTMENT_ID', 'DEPARTMENT_NAME', 'MANAGER_ID', 'LOCATION_ID'], dtype='object')

In [8]:
location = pd.read_csv('./data/location.csv')
location.head()

,LOCATION_ID,LOCATION_NAME,STATE,COUNTRY
0,1700,Ikeja,Lagos,Nigeria
1,1800,Ibadan,Oyo,Nigeria
2,1900,Kaduna,Kaduna,Nigeria
3,2000,PH,Rivers,Nigeria
4,2400,Warri,Delta,Nigeria


In [9]:
location.shape

(12, 4)

In [10]:
location.columns

Index(['LOCATION_ID', 'LOCATION_NAME', 'STATE', 'COUNTRY'], dtype='object')

Question 1. 
- Number of remote workers in each department i.e. staff working outside their department location excluding shipping staff?


In [59]:
sql1 = ''' SELECT department.DEPARTMENT_NAME, COUNT(*) AS Total_Remote_Worker
FROM department 
JOIN employee ON department.DEPARTMENT_ID = employee.DEPARTMENT_ID
WHERE employee.RESIDENCY_ID!=department.LOCATION_ID 
AND department.DEPARTMENT_NAME!='Shipping'
GROUP BY department.DEPARTMENT_ID
'''

sqldf(sql1)

,DEPARTMENT_NAME,Total_Remote_Worker
0,Senior_Executive,1
1,Marketing,1
2,Purchasing,4
3,IT,3


this query shows that no employe works remotly in Finance and Accounting departments!

Question 2.
- Create a manager information dimension table

In [5]:
sql2 = ''' 
CREATE TABLE Dim_manager (
  manager_ID INT IDENTITY,
  manager_Name NVARCHAR(10) NOT NULL,
  RESIDENCY_ID INT NOT NULL)

INSERT INTO Dim_manager VALUES
 (200.0, 'Abdi', 1800),
 (148.0, 'John', 1700)
select * from Dim_manager
'''
sqldf(sql2)

PandaSQLException: (sqlite3.OperationalError) near "INSERT": syntax error
[SQL:  
CREATE TABLE Dim_manager (
  manager_ID INT IDENTITY,
  manager_Name NVARCHAR(10) NOT NULL,
  RESIDENCY_ID INT NOT NULL)

INSERT INTO Dim_manager VALUES
 (200.0, 'Abdi', 1800),
 (148.0, 'John', 1700)
select * from Dim_manager
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

Question 3.
- Number of workers earning above the average salary in the organization

In [60]:
sql3 = ''' SELECT AVG(SALARY) as AVG_SALARY, COUNT(EMPLOYEE_ID) AS Number_of_workers
FROM employee WHERE SALARY > (SELECT AVG(SALARY) FROM employee)
'''

In [61]:
sqldf(sql3)

,AVG_SALARY,Number_of_workers
0,10500.8,20


In [26]:
sql33 = ''' SELECT DEPARTMENT_NAME, COUNT(EMPLOYEE_ID) AS Number_of_workers 
FROM department JOIN employee on department.DEPARTMENT_ID = employee.DEPARTMENT_ID
GROUP BY department.DEPARTMENT_ID HAVING employee.SALARY > AVG(SALARY)
'''
sqldf(sql33)

,DEPARTMENT_NAME,Number_of_workers
0,Marketing,2
1,Purchasing,6
2,Shipping,23
3,IT,5
4,Finance,6
5,Accounting,2


Question 4.
Create a new column in the employee table categorizing staff as either early hire or late based on the hire date i.e. if the hire date is greater or equal to the average hire date in the organization that should be late hire else early hire as this will assist in shares distribution.

In [59]:
employee["HIRE_DATE"] = pd.to_datetime(employee["HIRE_DATE"])
employee.head()


,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,HIRE_DATE,JOB_ID,SALARY,DEPARTMENT_ID,RESIDENCY_ID
0,198,Donald,OConnell,DOCONNEL,2007-06-21,SH_CLERK,2600,50,1500
1,199,Douglas,Grant,DGRANT,2008-01-13,SH_CLERK,2600,50,1500
2,200,Jennifer,Whalen,JWHALEN,2003-09-17,AD_ASST,4400,10,1700
3,201,Michael,Hartstein,MHARTSTE,2004-02-17,MK_MAN,13000,20,1000
4,202,Pat,Fay,PFAY,2005-08-17,MK_REP,6000,20,1800


In [62]:
employee['year'] = employee['HIRE_DATE'].dt.year
employee['month'] = employee['HIRE_DATE'].dt.month
employee['day'] = employee['HIRE_DATE'].dt.day

employee.head()

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,HIRE_DATE,JOB_ID,SALARY,DEPARTMENT_ID,RESIDENCY_ID,year,month,day
0,198,Donald,OConnell,DOCONNEL,2007-06-21,SH_CLERK,2600,50,1500,2007,6,21
1,199,Douglas,Grant,DGRANT,2008-01-13,SH_CLERK,2600,50,1500,2008,1,13
2,200,Jennifer,Whalen,JWHALEN,2003-09-17,AD_ASST,4400,10,1700,2003,9,17
3,201,Michael,Hartstein,MHARTSTE,2004-02-17,MK_MAN,13000,20,1000,2004,2,17
4,202,Pat,Fay,PFAY,2005-08-17,MK_REP,6000,20,1800,2005,8,17


In [64]:
avg = sqldf(''' SELECT cast(ROUND(AVG(year)) as int)||'-'||cast(ROUND(AVG(month)) as int)||'-'||cast(ROUND(AVG(day)) as int) FROM employee

''')
avg

,cast(ROUND(AVG(year)) as int)||'-'||cast(ROUND(AVG(month)) as int)||'-'||cast(ROUND(AVG(day)) as int)
0,2005-6-15


In [65]:
sql = '''
DECLARE @avg date, @status varchar

SET @avg = ( SELECT cast(ROUND(AVG(year)) as int)||'-'||cast(ROUND(AVG(month)) as int)||'-'||cast(ROUND(AVG(day)) as int) FROM employee
SET @status = (
    if (employee.HIRE_DATE >= @avg)
    @status = 'late_hire'
    else
    @status = 'early_hire'
)
SELECT * from employee
)

'''
sqldf(sql)

PandaSQLException: (sqlite3.OperationalError) near "DECLARE": syntax error
[SQL: 
DECLARE @avg date, @status varchar

SET @avg = ( SELECT cast(ROUND(AVG(year)) as int)||'-'||cast(ROUND(AVG(month)) as int)||'-'||cast(ROUND(AVG(day)) as int) FROM employee
SET @status = (
    if (employee.HIRE_DATE >= @avg)
    @status = 'late_hire'
    else
    @status = 'early_hire'
)
SELECT * from employee
)

]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [7]:
sql44 = '''SELECT EMPLOYEE_ID, FIRST_NAME, LAST_NAME, EMAIL, HIRE_DATE, JOB_ID, SALARY, DEPARTMENT_ID, RESIDENCY_ID ,
                            CASE
                                WHEN CAST(hire_date AS NUMERIC) >= (SELECT AVG(hire_date) FROM employee) THEN 'late hire'
                                ELSE 'early hire'
                            END AS hire_status
                        FROM employee
                        '''
sqldf(sql44)

PandaSQLException: (sqlite3.OperationalError) no such table: employee
[SQL: SELECT EMPLOYEE_ID, FIRST_NAME, LAST_NAME, EMAIL, HIRE_DATE, JOB_ID, SALARY, DEPARTMENT_ID, RESIDENCY_ID ,
                            CASE
                                WHEN CAST(hire_date AS NUMERIC) >= (SELECT AVG(hire_date) FROM employee) THEN 'late hire'
                                ELSE 'early hire'
                            END AS hire_status
                        FROM employee
                        ]
(Background on this error at: https://sqlalche.me/e/14/e3q8)